<a href="https://colab.research.google.com/github/mohittalwar/python/blob/main/PythonTutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Python Tutorial++

Addendum to [Python 3.5 Tutorial](https://drive.google.com/open?id=1B8IvfcO7Gt_lXT6a-Bbvpqe4BXi5SW4U)

In [1]:
!python3 --version
!pip3 install aiofiles aiohttp

Python 3.13.1


## 👫 Match

In [2]:
# 1. Take an expression and compare its value to successive patterns
def http_error(status: int) -> str:
    match status:
        case 400:
            return "Bad request"
        case 404:
            return "Not found"
        case 401 | 403:  # multiple literals
            return "Not allowed"
        case _:  # wildcard
            return "Something is wrong with the Internet"


# 2. Bind variables
def point_name(point: tuple[int, int]) -> str:
    match point:
        case (0, 0):
            return "Origin"
        case (0, y):
            return f"Y={y}"
        case (x, 0):
            return f"X={x}"
        case (x, y) if x == y:  # guard match
            return f"Y=X at {x}"
        case (x, y):
            return f"X={x}, Y={y}"
        case _:
            raise ValueError("Not a point")


## ⛳ Enum
Set of symbolic names bound to unique values

In [3]:
from enum import Enum, Flag, auto


class Color(Enum):
    RED = 1
    GREEN = 2
    BLUE = 3


class Days(Flag):
    MONDAY = 1
    TUESDAY = 2
    WEDNESDAY = 4
    THURSDAY = 8
    FRIDAY = 16
    SATURDAY = 32
    SUNDAY = 64
    WEEKEND = SATURDAY | SUNDAY


class AutoDays(Flag):
    MONDAY = auto()
    TUESDAY = auto()
    WEDNESDAY = auto()
    THURSDAY = auto()
    FRIDAY = auto()
    SATURDAY = auto()
    SUNDAY = auto()
    WEEKEND = SATURDAY | SUNDAY


for day in AutoDays.WEEKEND:
    print(type(day), day, day.name, day.value)

<flag 'AutoDays'> AutoDays.SATURDAY SATURDAY 32
<flag 'AutoDays'> AutoDays.SUNDAY SUNDAY 64


## 📈 Functional Programming


In [4]:
# 1. Decorators
from typing import Any, Callable

def print_args(fn: Callable) -> Callable:
    def decorated_fn(*args: Any, **kwargs: Any) -> Any:
        print(f"Positional args: {args}")
        print(f"Keyword args: {kwargs}")
        return fn(*args, **kwargs)

    return decorated_fn


@print_args  # Equivalent to add = print_args(add)
def add(a: int, b: int) -> int:
    return a + b


print(add(2, 3))

# 2. Helper Generators: map, filter

numbers = [1, 2, 3, 4, 5]
print(*map(lambda x: x * x, numbers))
print(*filter(lambda x: x % 2 == 0, numbers))

# 3. Functools

from functools import cache, partial, reduce


@cache
def factorial(n: int) -> int:
    return n * factorial(n - 1) if n > 0 else 1


print("Cache factorial(5):", factorial(5))
print("Reuse factorial(5):", factorial(6))

relu = partial(max, 0)
print(relu(-1), relu(1))


def factoriall(n: int) -> int:
    return reduce(lambda x, y: x * y, range(1, n + 1))


print(factoriall(5))

Positional args: (2, 3)
Keyword args: {}
5
1 4 9 16 25
2 4
Cache factorial(5): 120
Reuse factorial(5): 720
0 1
120


## 💾 Asyncio
For i/o-bound applications.

Concurrency via non-blocking io, managed by a single-threaded event-loop.

In [5]:
import aiofiles
import aiohttp
import asyncio
import random


async def coro1() -> int:
    for i in range(3):
        # await cedes control to the event loop; it can only be invoked from coros.
        await asyncio.sleep(random.random())
        print(f"Coroutine 1: {i}")
    return 1


async def coro2_helper() -> asyncio.Generator[int, None, None]:
    for i in range(3):
        await asyncio.sleep(random.random())
        yield i


async def coro2() -> int:
    async for i in coro2_helper():
        print(f"Coroutine 2: {i}")
    return 2


async def coro3_helper(session: aiohttp.ClientSession, file: aiofiles.tempfile.NamedTemporaryFile, url: str) -> int:
    await asyncio.sleep(random.random())
    try:
        async with session.get(url) as response:
            await file.write(await response.text())
            status = response.status
    except aiohttp.ClientError:
        status = -1

    print(f"Coroutine 3: {url}")
    return status


async def coro3() -> list[int]:
    async with (
        aiohttp.ClientSession() as session,
        aiofiles.tempfile.NamedTemporaryFile("w+") as file,
    ):
        helpers = [
            coro3_helper(
                session,
                file,
                f"https://www.randomnumberapi.com/api/v1.0/random?count={i + 1}",
            )
            for i in range(3)
        ]
        result = await asyncio.gather(*helpers)

        await file.seek(0)
        async for line in file:
            print(line, end="")

    return result


async def main() -> None:
    # Schedule all coroutines to run concurrently.
    results = await asyncio.gather(coro1(), coro2(), coro3())
    print(results)


def is_event_loop_running() -> bool:
    try:
        asyncio.get_running_loop()
        return True
    except RuntimeError:
        return False


assert is_event_loop_running()

# Schedule on the running event loop in Jupyter.
# Note: For standalone scripts, use asyncio.run(main()) to start the event loop.
await main()

AttributeError: module 'asyncio' has no attribute 'Generator'

## 🖥 Multiprocessing or ProcessPoolExecutor
For cpu-bound applications.

Concurrency via multiple processes, to bypass the GIL.

Note1: Combine asyncio and multiprocessesing with [aiomultiprocess](https://github.com/omnilib/aiomultiprocess).

Note2: Alternatively, use Ray for distributed programming.

In [25]:
import multiprocessing
from concurrent.futures import ProcessPoolExecutor


def cpu_task(n: int) -> int:
    return sum(i * i for i in range(n))

def main1() -> None:
    with multiprocessing.Pool() as pool:
        results = pool.map(cpu_task, [10**6, 10**6, 10**6])
    print(results)

def main2() -> None:
    with ProcessPoolExecutor() as executor:
        results = executor.map(cpu_task, [10**6, 10**6, 10**6])
    for result in results:
        print(result)

# main1()
# main2()

## 🏛 Libraries

In [26]:
# 1. collections: https://docs.python.org/3/library/collections.html

from collections import Counter, defaultdict, deque, namedtuple, OrderedDict
from typing import List, Tuple, Dict

letters = Counter("mississippi")
print(*letters, letters.total())
print(*letters.elements())
print(letters.most_common(2))

d: Dict[str, List[int]] = defaultdict(list)
for i, c in enumerate("mississippi"):
    d[c].append(i)
print(d)

queue = deque([1, 2, 3])
queue.appendleft(0)
print(queue.popleft())

Point = namedtuple("Point", ["x", "y"])
p = Point(1, 1)
print(f"{p.x=}, {p.y=}")

d = OrderedDict.fromkeys("abcde")
a = d.popitem()
print(*d)
e = d.popitem(last=False)
print(*d)
d.move_to_end("b")
print(*d)
d.move_to_end("b", last=False)
print(*d)


# 2. itertools: https://docs.python.org/3/library/itertools.html

from itertools import cycle, islice, product, starmap

# Infinators
print(*islice(cycle("ABCD"), 10))

# Terminators
print(*starmap(pow, [(1, 2), (2, 3), (3, 4), (4, 5)]))

# Combinators
print(*product("ABC", "123"))


# 3. pickle: serialize/unserialize python objects in binary

import pickle
import tempfile

idata: Dict[str, str] = {"name": "Mohit", "city": "Seattle"}

with tempfile.NamedTemporaryFile(prefix="tmp", suffix=".pkl", mode="w+b",) as file:
    pickle.dump(idata, file)
    file.seek(0)
    odata: Dict[str, str] = pickle.load(file)

assert idata == odata

m i s p 11
m i i i i s s s s p p
[('i', 4), ('s', 4)]
defaultdict(<class 'list'>, {'m': [0], 'i': [1, 4, 7, 10], 's': [2, 3, 5, 6], 'p': [8, 9]})
0
p.x=1, p.y=1
a b c d
b c d
c d b
b c d
A B C D A B C D A B
1 8 81 1024
('A', '1') ('A', '2') ('A', '3') ('B', '1') ('B', '2') ('B', '3') ('C', '1') ('C', '2') ('C', '3')


## 🎲 Miscellaneous

In [27]:
# 1. Walrus Operator: assign values to variables as part of a larger expression:
suffix: List[str] = []
word: List[str] = list("prefix_suffix")
while (c := word.pop()) != "_":
    suffix.append(c)
print("".join(word), "".join(reversed(suffix)))


# 2. F-strings: include the value of an expression inside a string using {}
# Optionally use '=' to include the expression text
# Optionally use ':' to provide a format specifier
two_thirds: float = 2 / 3
print(f"Super majority requires {two_thirds=:.2%}")


# 3. Super: Call methods of the parent class:
class A:
    def __init__(self) -> None:
        print("A")


class B(A):
    def __init__(self) -> None:
        print("B")


class C(B):
    def __init__(self) -> None:
        super().__init__()  # Same as super(C, self).__init__()
        print("C")


c = C()


# 4. Context Manager:
# . Call expression to obtain a context manager
# . Store the context manager’s .__enter__() and .__exit__() methods
# . Call .__enter__() and bind its return value to the optional target variable
# . Execute the with code block
# . Call .__exit__() when the with code block finishes
import os

with os.scandir(".") as entries:
    for entry in entries:
        print(entry.name, "->", entry.stat().st_size, "bytes")

prefix suffix
Super majority requires two_thirds=66.67%
B
C
PythonTutorial.ipynb -> 19168 bytes
README.md -> 102 bytes
.venv -> 256 bytes
app.py -> 23 bytes
.git -> 480 bytes
